In [2]:
# looking at the advanced stats for a single fight 
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

fights = requests.get('http://ufcstats.com/fight-details/037d2f426c916ffc').text
soupo = BeautifulSoup(fights, 'lxml')
data = []
for p in soupo.findAll('p', {"class":"b-fight-details__table-text"}):
    mystr=p.string
    mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
    if mystr != 'None':
        data.append(mystr)
    print(mystr)

None
None
0
0
28 of 60
7 of 30
46%
23%
29 of 62
7 of 30
0 of 6
0 of 0
0%
0%
0
0
0
0
0
0
None
None
0
0
14 of 38
5 of 23
36%
21%
15 of 40
5 of 23
0 of 4
0 of 0
0%
0%
0
0
0
0
0
0
None
None
0
0
14 of 22
2 of 7
63%
28%
14 of 22
2 of 7
0 of 2
0 of 0
0%
0%
0
0
0
0
0
0
None
None
28 of 60
7 of 30
46%
23%
20 of 51
3 of 23
3 of 3
2 of 4
5 of 6
2 of 3
19 of 51
7 of 30
9 of 9
0 of 0
0 of 0
0 of 0
None
None
14 of 38
5 of 23
36%
21%
8 of 32
2 of 18
2 of 2
2 of 3
4 of 4
1 of 2
14 of 38
5 of 23
0 of 0
0 of 0
0 of 0
0 of 0
None
None
14 of 22
2 of 7
63%
28%
12 of 19
1 of 5
1 of 1
0 of 1
1 of 2
1 of 1
5 of 13
2 of 7
9 of 9
0 of 0
0 of 0
0 of 0


In [7]:
# main loop for appending advanced stats
# NOTE: just showing 80 events for example, change x value for all UFC events (over 500)


def get_winner(input_link):
	linko = []
	fight_winner = requests.get(input_link).text
	soup_winner = BeautifulSoup(fight_winner,'lxml')

	for everylink in soup_winner.findAll('div',{"class":"b-fight-details__person"}):
		linky = re.sub(r'[\ \n]{2,}', '', str(everylink.text))

		if linky[0:1] == 'W':
			if '"' in linky:
				num = linky.index('"')
				linko.append(linky[1:num])
			else:
				linko.append(linky[1:])

	#See if its a title fight

	if 'Title Bout' in soup_winner.find('i',{"class":"b-fight-details__fight-title"}).text:
		linko.append('Title Bout')

	else:
		linko.append('Non-Title Bout')

	for img in soup_winner.findAll('img'):		

		if 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/fight.png' == img.get('src'):
			linko.append("FIGHT")
		elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/perf.png' == img.get('src'):
			linko.append("PERF")
		elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/ko.png' == img.get('src'):
			linko.append("KO")
		elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/sub.png' == img.get('src'):
			linko.append("SUB")
	if len(linko) == 4:
		linko[2] = linko[2] + " " + linko[3]
		linko.pop()
	
	elif 'Title' in linko[0]:
		linko.insert(0,'Draw')

	elif len(linko) == 2:
		linko.append(" ")
	return(linko)



def getnames(input_link):
	z = 0
	names =[]
	fight_detail_request = requests.get(input_link).text
	soup_details = BeautifulSoup(fight_detail_request,'lxml')
	for link in soup_details.findAll('p', {"class":"b-fight-details__table-text"}):
		if z < 2:
			#link = re.sub(r'[\ \n]{2,}', '', str(link.text)
			name = link.text[1:len(link.text) -2]
			names.append(name)
			z +=1
		else:
			break

	for link in soup_details.title.findAll('h2',{"class":"b-content__title"}):
		names.append(link.text)

	wclass = soup_details.find('div',{"class":"b-fight-details__fight-head"}).text
	wclass = re.sub(r'[\ \n]{2,}', '', str(wclass))
	if 'Title' in wclass:
		weight = wclass[4:wclass.index("Title") -1]
		names.append(weight)
	else:
		names.append(wclass)
	return(names)	

#print(getnames("http://www.ufcstats.com/fight-details/fae99a4089f8abb0"))
def getevents(input_link):
	fight_detail_request = requests.get(input_link).text
	soup_details = BeautifulSoup(fight_detail_request,'lxml')
	mylist = []
	myevent = soup_details.find('h2',{"class":"b-content__title"}).text
	myevent = re.sub(r'[\ \n]{2,}', '', str(myevent))
	mylist.append(myevent)
	for link in soup_details.findAll('li',{"class":"b-list__box-list-item"}):
		mystr = re.sub(r'[\ \n]{2,}', '', str(link.text))
		if 'Location' in mystr:
			mylist.append(mystr[9:])
		elif 'Date' in mystr:
			mylist.append(mystr[5:])
		else:
			mylist.append(mystr[11:])
	return(mylist)

print(getevents("http://www.ufcstats.com/event-details/4956f60b7fa57c1a"))

def getstats(input_link):
	fight_stats = []

	fight_stats.extend(getref(input_link))
	fight_stats.extend(get_winner(input_link))
	fight_link = requests.get(input_link).text
	soup_fight = BeautifulSoup(fight_link, 'lxml')
	for stats in soup_fight.findAll('p', {"class":"b-fight-details__table-text"}):
			mystr=stats.string
			mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
			fight_stats.append(mystr)

	if fight_stats[1] == '1':
		fight_stats += 70 *['N/A']
		for i in range(51,133):
			fight_stats.insert(i,"N/A")

	elif fight_stats[1] == '2':
		fight_stats += 54 *['N/A']
		for i in range(71,131):
			fight_stats.insert(i,"N/A")

	elif fight_stats[1] == '3':
		fight_stats += 36 *['N/A']
		for i in range(91,131):
			fight_stats.insert(i,"N/A")

	elif fight_stats[1] == '4':
		fight_stats += 16 *['N/A']
		for i in range(110,131):
			fight_stats.insert(i,"N/A")


	return(fight_stats)



#132-53 for round 1 finises


def getref(input_link):
	#txt is the long string that is returned from soup
	txt = []
	#ref_clean is the clean appened list of things we want to use
	ref_clean = []
	fight_link = requests.get(input_link).text
	soup_ref = BeautifulSoup(fight_link, 'lxml')
	for refs in soup_ref.findAll('p', {"class":"b-fight-details__text"}):
		mystr = refs.text
		mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
		txt.append(mystr) 

	#Parce through the online long string that is fighter details

	Method_num = txt[0].index('Round')
	ref_clean.append(txt[0][7:Method_num])

	Round_num = txt[0].index('Time')
	ref_clean.append(txt[0][Method_num+6:Round_num])

	Rndnum = txt[0].index('Time format')
	ref_clean.append(txt[0][Round_num+5:Rndnum])

	Time_num = txt[0].index('Referee')
	ref_clean.append(txt[0][Rndnum+12:Time_num])

	referee = txt[0][Time_num+8:]
	ref_clean.append(referee)

	ref_clean.append(txt[1][8:])
	return(ref_clean)



#MAIN LOOP
def main():
	#main soup for events page
	source = requests.get('http://www.ufcstats.com/statistics/events/completed?page=all').text
	soup = BeautifulSoup(source, 'lxml')
	x=0

	with open(r'C:\Users\brock\OneDrive\Documents\quick_join.csv','w', newline='') as file:
		writer = csv.writer(file)
		csvheader = ['FIGHTER_RED','FIGHER_BLUE','WEIGHT','EVENT','DATE','STATE_COUNTRY','ATTENDENCE','FINISH','ROUND_FINISHED','TIME_FINISHED','SCHEDUELED ROUNDS','REFEREE','METHOD_OF_FINISH',
				     'WINNER','TITLE_OR_NOT','BONUS','?','?','KD_RED','KD_BLUE','SIG_STR_RED','SIG_STR_BLUE','SIG_STR_PERCENT_RED','SIG_STR_PERCENT_BLUE',
				     'TOTAL_STR_RED','TOTAL_STR_BLUE','TD_RED','TD_BLUE','TD_PERCENT_RED','TD_PERCENT_BLUE','SUB_ATT_RED','SUB_ATT_BLUE','PASSES_RED',
				     'PASSES_BLUE','REV_RED','REV_BLUE','?','?','RND1_KD_RED','RND1_KD_BLUE','RND1_SIG_STR_RED','RND1_SIG_STR_BLUE','RND1_SIG_STR_PERC_RED',
				     'RND1_SIG_STR_PERC_BLUE','RND1_TOTAL_STR_RED','RND1_TOTAL_STR_BLUE','RND1_TD_RED','RND1_TD_BLUE','RND1_TD_PERC_RED','RND1_TD_PERC_BLUE',
				     'RND1_SUB_ATT_RED','RND1_SUB_ATT_BLUE','RND1_PASSES_RED','RND1_PASSES_BLUE','RND1_REV_RED','RND1_REV_BLUE','?','?','RND2_KD_RED','RND2_KD_BLUE',
				     'RND2_SIG_STR_RED','RND2_SIG_STR_BLUE','RND2_SIG_STR_PERC_RED',
				     'RND2_SIG_STR_PERC_BLUE','RND2_TOTAL_STR_RED','RND2_TOTAL_STR_BLUE','RND2_TD_RED','RND2_TD_BLUE','RND2_TD_PERC_RED','RND2_TD_PERC_BLUE',
				     'RND2_SUB_ATT_RED','RND2_SUB_ATT_BLUE','RND2_PASSES_RED','RND2_PASSES_BLUE','RND2_REV_RED','RND2_REV_BLUE','?','?','RND3_KD_RED','RND3_KD_BLUE',
				     'RND3_SIG_STR_RED','RND3_SIG_STR_BLUE','RND3_SIG_STR_PERC_RED',
				     'RND3_SIG_STR_PERC_BLUE','RND3_TOTAL_STR_RED','RND3_TOTAL_STR_BLUE','RND3_TD_RED','RND3_TD_BLUE','RND3_TD_PERC_RED','RND3_TD_PERC_BLUE',
				     'RND3_SUB_ATT_RED','RND3_SUB_ATT_BLUE','RND3_PASSES_RED','RND3_PASSES_BLUE','RND3_REV_RED','RND3_REV_BLUE','?','?','RND4_KD_RED','RND4_KD_BLUE',
				     'RND4_SIG_STR_RED','RND4_SIG_STR_BLUE','RND4_SIG_STR_PERC_RED',
				     'RND4_SIG_STR_PERC_BLUE','RND4_TOTAL_STR_RED','RND4_TOTAL_STR_BLUE','RND4_TD_RED','RND4_TD_BLUE','RND4_TD_PERC_RED','RND4_TD_PERC_BLUE',
				     'RND4_SUB_ATT_RED','RND4_SUB_ATT_BLUE','RND4_PASSES_RED','RND4_PASSES_BLUE','RND4_REV_RED','RND4_REV_BLUE','?','?','RND5_KD_RED','RND5_KD_BLUE',
				     'RND5_SIG_STR_RED','RND5_SIG_STR_BLUE','RND5_SIG_STR_PERC_RED',
				     'RND5_SIG_STR_PERC_BLUE','RND5_TOTAL_STR_RED','RND5_TOTAL_STR_BLUE','RND5_TD_RED','RND5_TD_BLUE','RND5_TD_PERC_RED','RND5_TD_PERC_BLUE',
				     'RND5_SUB_ATT_RED','RND5_SUB_ATT_BLUE','RND5_PASSES_RED','RND5_PASSES_BLUE','RND5_REV_RED','RND5_REV_BLUE','?','?','SIG_STR_TOTAL_RED','SIG_STR_TOTAL_BLUE',
				     'SIG_STR_PERC_RED','SIG_STR_PERC_BLUE','HEAD_STR_RED','HEAD_STR_BLUE','BODY_STR_RED','BODY_STR_BLUE','LEG_STR_RED','LEG_STR_BLUE','DISTANCE_RED','DISTANCE_BLUE',
				     'CLINCH_RED','CLICH_BLUE','GROUD_RED','GROUND_BLUE','?','?','RND1_SIG_STR_RED','RND1_SIG_STR_BLUE','RND1_SIG_STR_PERC_RED','RND1_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND1_HEAD_STR_BLUE','RND1_BODY_STR_RED','RND1_BODY_STR_BLUE','RND1_LEG_STR_RED','RND1_LEG_STR_BLUE','RND1_DISTANCE_RED','RND1_DISTANCE_BLUE',
				     'RND1_CLINCH_RED','RND1_CLINCH_BLUE','RND1_GROUND_RED','RND1_GOUND_BLUE','?','?','RND2_SIG_STR_RED','RND2_SIG_STR_BLUE','RND2_SIG_STR_PERC_RED','RND2_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND2_HEAD_STR_BLUE','RND2_BODY_STR_RED','RND2_BODY_STR_BLUE','RND2_LEG_STR_RED','RND2_LEG_STR_BLUE','RND2_DISTANCE_RED','RND2_DISTANCE_BLUE',
				     'RND2_CLINCH_RED','RND2_CLINCH_BLUE','RND2_GROUND_RED','RND2_GOUND_BLUE','?','?','RND3_SIG_STR_RED','RND3_SIG_STR_BLUE','RND3_SIG_STR_PERC_RED','RND3_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND3_HEAD_STR_BLUE','RND3_BODY_STR_RED','RND3_BODY_STR_BLUE','RND3_LEG_STR_RED','RND3_LEG_STR_BLUE','RND3_DISTANCE_RED','RND3_DISTANCE_BLUE',
				     'RND3_CLINCH_RED','RND3_CLINCH_BLUE','RND3_GROUND_RED','RND3_GOUND_BLUE','?','?','RND4_SIG_STR_RED','RND4_SIG_STR_BLUE','RND4_SIG_STR_PERC_RED','RND4_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND4_HEAD_STR_BLUE','RND4_BODY_STR_RED','RND4_BODY_STR_BLUE','RND4_LEG_STR_RED','RND4_LEG_STR_BLUE','RND4_DISTANCE_RED','RND4_DISTANCE_BLUE',
				     'RND4_CLINCH_RED','RND4_CLINCH_BLUE','RND4_GROUND_RED','RND4_GOUND_BLUE','?','?','RND5_SIG_STR_RED','RND5_SIG_STR_BLUE','RND5_SIG_STR_PERC_RED','RND5_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND5_HEAD_STR_BLUE','RND5_BODY_STR_RED','RND5_BODY_STR_BLUE','RND5_LEG_STR_RED','RND5_LEG_STR_BLUE','RND5_DISTANCE_RED','RND5_DISTANCE_BLUE',
				     'RND5_CLINCH_RED','RND5_CLINCH_BLUE','RND5_GROUND_RED','RND5_GOUND_BLUE']


		writer.writerow(csvheader)
		print('writing csv')


	for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
		print(x)
        #set x to a value depending on how many events for the dataset

		x = x + 1
		if x<80:
				#check if its an event details link
			if "event-details" in link.get('href'):
				print(link.get('href'))
				#soup2 is the event details link
				#souping up the event link
				source2 = requests.get(link.get('href')).text
				soup2 = BeautifulSoup(source2, 'lxml')

				csvdata = getevents(link.get('href'))
				#looping through event link to get every fight-details link
				for links in soup2.findAll('a', attrs={'href': re.compile("^http://")}):

					if "fight-details" in links.get('href'):
						source3 = requests.get(links.get('href')).text
						soup3 = BeautifulSoup(source3, 'lxml')
						csvdata = getnames(links.get('href'))
						csvdata.extend(getevents(link.get('href')))
						csvdata.extend(getstats(links.get('href')))

						print(getevents(link.get('href')))

						with open(r'C:\Users\brock\OneDrive\Documents\quick_join.csv','a', newline='') as file:
							writer = csv.writer(file)
							#csv row is the stats list
							writer.writerow(csvdata)
							print('writing csv')
													
							
		else:
			break


main()

['UFC 205: Alvarez vs McGregor', 'November 12, 2016', 'New York City, New York, USA', '20,427']
writing csv
0
1
2
http://www.ufcstats.com/event-details/fc9a9559a05f2704
3
http://www.ufcstats.com/event-details/33b2f68ef95252e0
['UFC Fight Night: Felder vs. Hooker', 'February 22, 2020', 'Auckland, New Zealand', '']
writing csv
['UFC Fight Night: Felder vs. Hooker', 'February 22, 2020', 'Auckland, New Zealand', '']
writing csv
['UFC Fight Night: Felder vs. Hooker', 'February 22, 2020', 'Auckland, New Zealand', '']
writing csv
['UFC Fight Night: Felder vs. Hooker', 'February 22, 2020', 'Auckland, New Zealand', '']
writing csv
['UFC Fight Night: Felder vs. Hooker', 'February 22, 2020', 'Auckland, New Zealand', '']
writing csv
['UFC Fight Night: Felder vs. Hooker', 'February 22, 2020', 'Auckland, New Zealand', '']
writing csv
['UFC Fight Night: Felder vs. Hooker', 'February 22, 2020', 'Auckland, New Zealand', '']
writing csv
['UFC Fight Night: Felder vs. Hooker', 'February 22, 2020', 'Auckla

In [30]:
#importing required CSVs and renaming a few column errors, also setting dataset display widths

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
advanced = pd.read_csv(r'C:\Users\brock\OneDrive\Documents\quick_join.csv')
advanced.rename({'TITLE_OR_NOT': 'METHOD_OF_FINISH', 'METHOD_OF_FINISH': 'WINNER','WINNER':'TITLE_OR_NOT'}, axis=1)
advanced

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
0,Paul Felder,Dan Hooker,Lightweight Bout,UFC Fight Night: Felder vs. Hooker,"February 22, 2020","Auckland, New Zealand",NaN,Decision - Split,5,5:00,5 Rnd (5-5-5-5-5),Marc Goddard,David Lethaby47 - 48.Howard Hughes47 - 48.Barr...,Dan Hooker,Non-Title Bout,FIGHT,None,None,0,0,11

In [31]:
# All duplicated columns with the bonus as None have column formatting issue, columns are shifted to the right by 1.

p1 = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')].iloc[:,0:16]
p1

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS


In [32]:
# Grabing the 2nd half of the data and shifting it over 1 column to the left to fix the formatting issue
p2 = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')].iloc[:,16:].shift(1,axis='columns',fill_value = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')]['?.1'])
p2

ValueError: could not broadcast input array from shape (0) into shape (0,1)

In [33]:
#join the 2 halfs together
df1 = pd.concat([p1,p2], sort = False, axis = 1)
df1

NameError: name 'p2' is not defined

In [34]:
#grabbing the duplicated values with proper column formatting
p3 = advanced[(advanced.duplicated()) & (advanced['BONUS'] != 'None')]
p3

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE


In [35]:
# adding them to the complete dataset containing all duplicated values
df1 = pd.concat([df1,p3])
df1

NameError: name 'df1' is not defined

In [36]:
# filling NAs as 0
advanced.fillna(value = 0,inplace = True)

In [37]:
# dropping all duplicated items then adding our corrected dataset 
advanced.drop_duplicates(keep = False,inplace = True)
advanced = pd.concat([advanced , df1])

NameError: name 'df1' is not defined

In [38]:
#Appears to be an issue with some columns being shifted from row 138 and onward (looking to fix this in the main loop in the future)
s1 = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False)].iloc[:,0:137]
s1

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12
1,Jimmy Crute,Michal Oleksiejczuk,Light Heavyweight Bout,UFC Fight Night: Felder vs. Hooker,"February 22, 2020","Auckland, New Zealand",0,Submission,1,3:29,3 Rnd (5-5-5),John Sharp,Kimura From Side Control,Jimmy Crute,Non-Title Bout,PERF,None,None,0,0,2 of 8,3 of 6,25%,50%,13 of 23,7 of 10,8 of 10,0 of 0,80%,0%,1,0,4,0,0,0,None,None,0,0,2 of 8,3 of 6,25%,50%,13 of 23,7 of 10,8 of 10,0 of 0,80%,0%,1,0,4,0,0,0,None,None,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,Ben Sosoli,Marcos Rogerio de Lima,Heavyweight Bout,UFC Fight Night: Felder vs. Hooker,"February 22, 2020","Auckland, New Zealand",0,KO/TKO,1,1:28,3 Rnd (5-5-5),Neil Swailes,Punch to Head At Distance,Marcos Rogerio de Lima,Non-Title Bout,,None,None,0,1,5 of 17,19 of 34,29%,55%,5 of 17,19 of 34,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,0,1,5 of 17,19 of 34,29%,55%,5 of 17,19 of 34,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,Kevin Aguilar,Zubaira Tukhugov,Featherweight Bout,UFC Fight Night: Felder vs. Hooker,"February 22, 2020","Auckland, New Zealand",0,KO/TKO,1,3:21,3 Rnd (5-5-5),Marc Goddard,Punches to Head On Ground,Zubaira Tukhugov,Non-Title Bout,,None,None,0,2,9 of 28,28 of 36,32%,77%,9 of 28,28 of 37,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,0,2,9 of 28,28 of 36,32%,77%,9 of 28,28 of 37,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,Callan Potter,Kenan Song,Welterweight Bout,UFC Fight Night: Felder vs. Hooker,"February 22, 2020","Auckland, New Zealand",0

In [39]:
#shifting the data so the columns are in the proper loations
s2 = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False)].iloc[:,137:].shift(-2,axis =1 )
s2

,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
1,0,2 of 8,3 of 6,25%,50%,2 of 7,3 of 6,0 of 1,0 of 0,0 of 0,0 of 0,2 of 6,3 of 6,0 of 2,0 of 0,0 of 0,0 of 0,None,None,2 of 8,3 of 6,25%,50%,2 of 7,3 of 6,0 of 1,0 of 0,0 of 0,0 of 0,2 of 6,3 of 6,0 of 2,0 of 0,0 of 0,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
3,0,5 of 17,19 of 34,29%,55%,4 of 16,15 of 30,1 of 1,2 of 2,0 of 0,2 of 2,5 of 17,18 of 33,0 of 0,0 of 0,0 of 0,1 of 1,None,None,5 of 17,19 of 34,29%,55%,4 of 16,15 of 30,1 of 1,2 of 2,0 of 0,2 of 2,5 of 17,18 of 33,0 of 0,0 of 0,0 of 0,1 of 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
5,0,9 of 28,28 of 36,32%,77%,4 of 22,26 of 34,3 of 4,2 of 2,2 of 2,0 of 0,9 of 28,15 of 22,0 of 0,0 of 0,0 of 0,13 of 14,None,None,9 of 28,28 of 36,32%,77%,4 of 22,26 of 34,3 of 4,2 of 2,2 of 2,0 of 0,9 of 28,15 of 22,0 of 0,0 of 0,0 of 0,13 of 14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
8,0,7 of 15,17 of 35,46%,48%,6 of 14,12 of 29,1 of 1,5 of 6,0 of 0,0 of 0,5 of 12,14 of 29,2 of 3,2 of 5,0 of 0,1 of 1,None,None,7 of 15,17 of 35,46%,48%,6 of 14,12 of 29,1 of 1,5 of 6,0 of 0,0 of 0,5 of 12,14 of 29,2 of 3,2 of 5,0 of 0,1 of 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
11,0,8 of 18,5 of 11,44%,45%,8 of 18,4 of 9,0 of 0,1 of 2,0 of 0,0 of 0,8 of 17,5 of 11,0 of 0,0 of 0,0 of 1,0 of 0,None,None,8 of 18,5 of 11,44%,45%,8 of 18,4 of 9,0 of 0,1 of 2,0 of 0,0 of 0,8 of 17,5 of 11,0 of 0,0 of 0,0 of 1,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
12,0,11 of 27,13 of 31,40%,41%,4 of 19,11 of 27,3 of 4,1 of 3,4 of 4,1 of 1,11 of 27,12 of 29,0 of 0,0 of 1,0 of 0,1 of 1,None,None,11 of 27,13 of 31,40%,41%,4 of 19,11 of 27,3 of 4,1 of 3,4 of 4,1 of 1,11 of 27,12 of 29,0 of 0,0 of 1,0 of 0,1 of 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [40]:
#adding the 2 peices together
df2 = pd.concat([s1,s2], axis = 1)
df2

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
1,Jimmy Crute,Michal Oleksiejczuk,Light Heavyweight Bout,UFC Fight Night: Felder vs. Hooker,"February 22, 2020","Auckland, New Zealand",0,Submission,1,3:29,3 Rnd (5-5-5),John Sharp,Kimura From Side Control,Jimmy Crute,Non-Title Bout,PERF,None,None,0,0,2 of 8,3 of 6,25%,50%,13 o

In [41]:
advanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Columns: 244 entries, FIGHTER_RED to RND5_GOUND_BLUE
dtypes: float64(32), int64(17), object(195)
memory usage: 47.9+ KB


In [42]:
# removing all incorrectly formatted columns and then adding our corrected dataset
advanced = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False) == False]
advanced = pd.concat([advanced,df2])

In [43]:
advanced[advanced['WINNER'] == 'Draw']

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE


In [44]:
# similar column formation issue, creating corrected versions of the data
t1 = advanced[advanced['GROUND_BLUE'] == 'None'].iloc[:,0:137]
t1

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12


In [45]:
t2 = advanced[advanced['GROUND_BLUE'] == 'None'].iloc[:,137:].shift(1,axis=1)

In [46]:
df3 = pd.concat([t1,t2],axis=1)


In [47]:
df3

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE


In [48]:
advanced = advanced[advanced['GROUND_BLUE'] != 'None']

In [49]:
advanced = pd.concat([advanced,df3])
advanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 15
Columns: 244 entries, FIGHTER_RED to RND5_GOUND_BLUE
dtypes: float64(32), int64(17), object(195)
memory usage: 47.9+ KB


In [50]:
# removing unneeded columns
for i in advanced.columns:
    if i[0] == "?" or i[0:3] == 'Unn':
        print(i)
        advanced.drop(i,axis = 1,inplace=True)

?
?.1
?.2
?.3
?.4
?.5
?.6
?.7
?.8
?.9
?.10
?.11
?.12
?.13
?.14
?.15
?.16
?.17
?.18
?.19
?.20
?.21
?.22
?.23


In [51]:
#reseting the index 
advanced.reset_index(inplace = True)

In [52]:
advanced

,index,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
0,0,Paul Felder,Dan Hooker,Lightweight Bout,UFC Fight Night: Felder vs. Hooker,"February 22, 2020","Auckland, New Zealand",0,Decision - Split,5,5:00,5 Rnd (5-5-5-5-5),Marc Goddard,David Lethaby47 - 48.Howard Hughes47 - 48.Barr...,Dan Hooker,Non-Title Bout,FIGHT,0,0,110 of 230,122 of 221,47%,55%,119 of 243,133 of 234,0 of 1,1 of 5,0%,20%,0,0,0,1,0,0,0,0,16 of 42,24 of 46,38%,52%

In [53]:
#creating the loser stat, if the fighter isn't a winner or the fight wasn't a draw, he was a loser
advanced['LOSER'] = 0
count = 0
for i in advanced[['FIGHTER_RED','FIGHER_BLUE','WINNER']].values:
    if i[0] not in i[2]:
        advanced['LOSER'].loc[count] = i[0]
    elif i[1] not in i[2]:
        advanced['LOSER'].loc[count] = i[1]
    else:
        advanced['LOSER'].loc[count] = 'Draw'
        
    print(count)
        
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [54]:
advanced[advanced['LOSER'] == 'David Abbot']

,index,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE,LOSER


In [55]:
#fill NAs as 0
advanced.fillna(value = 0,inplace = True)

In [56]:
advanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Columns: 222 entries, index to LOSER
dtypes: float64(32), int64(18), object(172)
memory usage: 43.5+ KB


In [58]:
#writing to csv
advanced.to_csv(r'C:\Users\brock\OneDrive\Documents\UFC_Fight_Stats2.csv', index = None, header=True)